In [81]:
### 개인 컴퓨터에서 설정해야하는 부분###

file_path='/Users/gimseung-yeon/연습/annotation/후처리작업/admin.jsonl 10'  #doccano에서 export한 jsonfile
odd_annotation='/Users/gimseung-yeon/연습/annotation/후처리작업/이상한태깅.txt' #relation 및 entity 이상한거 보는 파일
new_file_path = '/Users/gimseung-yeon/연습/annotation/후처리작업/탑재1.json' #제일마지막에 생성할 파일

오류 = [['대지미사일', ['3천t'], '무게'],  #entity 이상한거 확인하면 [잘못된라벨, [대상단어], 올바른라벨]로 표기
       ['함포', ['20㎜ 포', '20㎜ 함포', '20mm 함포', '20㎜함포', '20mm 포', '20㎜ 포'], 'CIWS'],
       ['윤영하급', ['유도탄고속함', '유도탄 고속함'], '기타'],
       ['구형참수리급', ['참수리-286호'], '신형참수리급'],
       ['헬기', ['링스'], '항공기']]


In [78]:
import json
함급 = ["정조대왕급", "장보고급", "도산안창호급",  "한산도급", "손원일급","신형참수리급", "소양급", "항공기", "돌고래급", "광개토대왕급", "충무공이순신급", "세종대왕급", "울산급", "인천급", "대구급", "포항급", "동해급", "독도급", "운봉급", "고준봉급", "천왕봉급", "원산급", "남포급", "강경급", "양양급", "윤영하급", "구형참수리급", "솔개-II급", "무레나급", "천지급", "청해진급", "평택급", "통영급", "다도해급", "신세기급", "신천지급"]

rule = {
    '최대속력': [함급, ['속력']],
    '탑재하다': [함급, ['대공미사일', '대지미사일', '대함미사일', '함포', 'CIWS', '어뢰', '레이더', '상륙장갑차', '항공기']],
    '길이를 가지다': [함급, ['길이']],
    '폭을 가지다': [함급, ['길이']],
    '참가하다': [함급, ['훈련']],
    '진수되다': [함급, ['날짜']],
    '퇴역하다': [함급, ['날짜']],
    '배수톤수': [함급, ['무게']],
    '승조원수': [함급, ['인원']],
    '취역하다': [함급, ['날짜']]
}


In [79]:
#relation 검증용 / 파일보고 이상한부분은 도카노에서 직접 수정하기
with open(file_path, 'r', encoding='utf-8') as f:
    with open(odd_annotation, 'w', encoding='utf-8') as d:
    
        for line in f:
            data_dict = json.loads(line)  # JSON 형식의 문자열을 파이썬 사전으로 변환
            for relation in data_dict.get('relations', []):
                rule_key = relation.get('type')
                if rule_key in rule:
                    relation_head, relation_tail = relation.get('from_id'), relation.get('to_id')
                    head_label = next((entity['label'] for entity in data_dict.get('entities', []) if entity['id'] == relation_head), '')
                    tail_label = next((entity['label'] for entity in data_dict.get('entities', []) if entity['id'] == relation_tail), '')
                    
                    if (head_label not in rule[rule_key][0]) or (tail_label not in rule[rule_key][1]):
                        odd_sentence=str(data_dict['id']+1)+' '+data_dict['text']+' '+rule_key+ ' '+head_label+' '+tail_label+'\n'
                        d.write(odd_sentence)

In [82]:
#entity 검증용 / 여기까지했으면 저장된 파일보고 제일위의 '오류' 정의해주기

entity = {}

with open(file_path, 'r', encoding='utf-8') as f:
    
    for line in f:
        data_dict = json.loads(line)  # JSON 형식의 문자열을 파이썬 사전으로 변환
        sentence = data_dict['text']
        
        for entity_info in data_dict.get('entities', []):
            entity_key = entity_info.get('label')
            entity_labeled = sentence[entity_info['start_offset']:entity_info['end_offset']]
            
            if entity_key in entity:
                if entity_labeled not in entity[entity_key]:
                    entity[entity_key].append(entity_labeled)
            else:
                entity[entity_key] = [entity_labeled]

# entity 딕셔너리 출력
print(entity)
with open(odd_annotation, 'w', encoding='utf-8') as d:
    json.dump(entity, d, ensure_ascii=False, indent=4)


{'도산안창호급': ['도산안창호함', '안무함', '신채호함', '도산 안창호함', '도산 안창호 잠수함', '도산 안창호 함'], '대지미사일': ['잠수함발사탄도미사일', 'SLBM', '해성-3', '천룡', '해성Ⅲ', '해성Ⅱ', '현무 4-1', '현무 4-2', '현무 4-4', '해성-2', '현무 2B', '천검', '현무4-4', '잠수함 발사 탄도미사일', '해룡', '해성', '현무 2-B', '현무', 'SLMB', '현무-2B', '현무-2C', '해성-Ⅱ', '해성 Ⅲ', '수중발사 탄도미사일', '현무2-B', '현무Ⅳ-4', '수중발사탄도미사일', '3천t', 'SLAM-ER', '해성 III', '현무-4', '현무4', '현무-2'], '통영급': ['통영함'], '속력': ['21노트', '39㎞/h', '30노트', '시속 750㎞', '907㎞/h', '40노트', '시속 74㎞', '시속 37㎞', '20kts', '29노트', '290㎞/h', '41노트', '시속 75km', '시속 약 55㎞', '74km/h', '20노트', '55㎞/h', '시속 55㎞', '시속 83㎞', '74㎞/h', '23노트', '시속 55.56㎞', '24노트', '44km/h', '마하 0.85', '45노트', '시속 250km', '시속 37㎞h', '시속 55.5km', '시속 309㎞'], '광개토대왕급': ['을지문덕함', '광개토대왕함', '양만춘함', 'DDH973'], '무게': ['3200t', '3천t', '2800톤', '3000t', '3천200t', '7600t', '2300t', '4400t', '1800t', '1만4500t', '3200톤', '7600톤', '4400톤', '4만4천t', '3,000톤', '2500t', '1500t', '1000t', '1,500톤', '1,000톤', '7천600t', '4천400t', '200t', '1800톤', '250톤', '2천800t', '300톤', 

In [67]:
#엔티티 자동으로 바꿔줌

with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

new_list = []

for line in lines:
    data_dict = json.loads(line)  # JSON 형식의 문자열을 파이썬 사전으로 변환
    sentence = data_dict['text']

    new_entities = []
    for entity_info in data_dict.get('entities', []):
        entity_key = entity_info.get('label')
        entity_labeled = sentence[entity_info['start_offset']:entity_info['end_offset']]

        for error in 오류:
            if entity_key == error[0]:
                if entity_labeled in error[1]:
                    # 수정된 레이블로 교체
                    entity_key = error[2]
                    break

        new_entities.append({
            'id': entity_info['id'],
            'label': entity_key,
            'start_offset': entity_info['start_offset'],
            'end_offset': entity_info['end_offset']
        })

    # 수정된 entities로 업데이트
    data_dict['entities'] = new_entities
    new_list.append(data_dict)
    # 수정된 데이터를 출력하거나 다른 작업을 수행

with open(new_file_path, 'w', encoding='utf-8') as f:
    for i in new_list:
        f.write(json.dumps(i, ensure_ascii=False) + '\n')  # JSON 형식으로 저장
